In [1]:
import glob
import json
import os
import csv
import zstandard as zstd
import io

KEYWORDS = [
    "Capitol",
    "January 6",
    "riot",
    "insurrection",
    "election fraud",
    "stop the steal"
]

SELECTED_COLUMNS = [
    "id", "author", "created_utc", "title", "selftext", "score",
    "num_comments", "subreddit", "permalink", "author_flair_text"
]

In [2]:
def is_relevant(obj, keywords=KEYWORDS):
    text = ""
    for field in ["title", "selftext", "body"]:
        if field in obj and obj[field]:
            text += " " + str(obj[field])
    text = text.lower()
    return any(keyword.lower() in text for keyword in keywords)

def process_zst_files(input_folder, output_csv_zst):
    zst_files = glob.glob(os.path.join(input_folder, '*.zst'))
    if not zst_files:
        print("No .zst files found in the folder.")
        return

    temp_csv = output_csv_zst + '.tmp'
    
    with open(temp_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=SELECTED_COLUMNS, extrasaction='ignore')
        writer.writeheader()  # Write header once

        for zst_file in zst_files:
            print(f"Processing file: {zst_file}")
            with open(zst_file, 'rb') as f:
                dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
                with dctx.stream_reader(f) as reader:
                    text_stream = io.TextIOWrapper(reader, encoding='utf-8')
                    for line in text_stream:
                        try:
                            obj = json.loads(line)
                        except Exception as e:
                            print(f"Error decoding line in {zst_file}: {e}")
                            continue
                        
                        if not is_relevant(obj):
                            continue

                        filtered_obj = {col: obj.get(col, None) for col in SELECTED_COLUMNS}
                        writer.writerow(filtered_obj)

    with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
        cctx = zstd.ZstdCompressor()
        cctx.copy_stream(input_f, output_f)

    os.remove(temp_csv)
    print(f"Compressed CSV written to {output_csv_zst}")

In [3]:
input_folder = 'submissions'
output_csv_zst = 'submissions.csv.zst'
process_zst_files(input_folder, output_csv_zst)

Processing file: submissions/RS_2020-12.zst
Processing file: submissions/RS_2021-01.zst
Processing file: submissions/RS_2021-02.zst
Compressed CSV written to submissions.csv.zst


In [4]:
import pandas as pd
import zstandard as zstd
import io

def read_csv_zst(file_path):
    with open(file_path, 'rb') as compressed_file:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(compressed_file) as reader:
            return pd.read_csv(io.BytesIO(reader.read()))

# Usage
df = read_csv_zst("submissions.csv.zst")

In [5]:
df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
0,k48tky,MakeTotalDestr0i,1606780884,copy pasta about opioid crisis and america dec...,One of the more vivid images that Americans ha...,10,11,weirdcollapse,/r/weirdcollapse/comments/k48tky/copy_pasta_ab...,NaN
1,k48toq,daveto,1606780892,(National Review:) Trump Election Fraud Allega...,NaN,2,0,TheIntercept,/r/TheIntercept/comments/k48toq/national_revie...,NaN
2,k48u1e,LeagueOfRiotBot,1606780921,Patch 10.25 Preview [RiotRayYonggi],NaN,1,0,leagueofriot,/r/leagueofriot/comments/k48u1e/patch_1025_pre...,NaN
3,k48uf7,LeagueOfRiotBot,1606780952,Patch 10.25 Preview [RiotRayYonggi],NaN,1,0,leagueofriot,/r/leagueofriot/comments/k48uf7/patch_1025_pre...,NaN
4,k48uq3,FFBot,1606780979,"Official: [WDIS Flex] - Mon Evening, 11/30/2020",**Post your Who Do I Start (WDIS) questions he...,5,22,fantasyfootball,/r/fantasyfootball/comments/k48uq3/official_wd...,NaN


In [10]:
import pandas as pd

subreddits = [
    "politics",
    "Ask_Politics",
    "conspiracy",
    "QAnon",
    "Conservative",
    "liberal"
]

In [11]:
filtered_df = df[df["subreddit"].isin(subreddits)]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
17,k492ay,mintaphil,1606781641,"At the White House, Capitol and Supreme Court,...",NaN,208,10,politics,/r/politics/comments/k492ay/at_the_white_house...,NaN
34,k49h1t,[deleted],1606782935,"At the White House, Capitol and Supreme Court,...",[deleted],7,2,politics,/r/politics/comments/k49h1t/at_the_white_house...,NaN
36,k49j80,TransFattyAcid,1606783124,Sidney Powell Lawsuit Claims Potential Electio...,NaN,0,25,Conservative,/r/Conservative/comments/k49j80/sidney_powell_...,NaN
41,k49mrl,LexoSir,1606783444,"General McInerney talks Insurrection Act, must...",NaN,0,2,conspiracy,/r/conspiracy/comments/k49mrl/general_mcinerne...,NaN
44,k49n6m,LexoSir,1606783483,General McInerney talks Insurrection Act,NaN,1,0,Conservative,/r/Conservative/comments/k49n6m/general_mciner...,Moderate Conservative


In [12]:
filtered_df = filtered_df[filtered_df["author"] != "[deleted]"]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
17,k492ay,mintaphil,1606781641,"At the White House, Capitol and Supreme Court,...",NaN,208,10,politics,/r/politics/comments/k492ay/at_the_white_house...,NaN
36,k49j80,TransFattyAcid,1606783124,Sidney Powell Lawsuit Claims Potential Electio...,NaN,0,25,Conservative,/r/Conservative/comments/k49j80/sidney_powell_...,NaN
41,k49mrl,LexoSir,1606783444,"General McInerney talks Insurrection Act, must...",NaN,0,2,conspiracy,/r/conspiracy/comments/k49mrl/general_mcinerne...,NaN
44,k49n6m,LexoSir,1606783483,General McInerney talks Insurrection Act,NaN,1,0,Conservative,/r/Conservative/comments/k49n6m/general_mciner...,Moderate Conservative
47,k49q05,digitalcleaner,1606783734,"Election Fraud perpetrated by China, Russia, I...",NaN,1,2,politics,/r/politics/comments/k49q05/election_fraud_per...,NaN


In [13]:
import pandas as pd
import zstandard as zstd
import os

output_csv_zst = "submissions.csv.zst"

temp_csv = "filtered_subreddits.csv"
filtered_df.to_csv(temp_csv, index=False, encoding="utf-8")

with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
    cctx = zstd.ZstdCompressor()
    cctx.copy_stream(input_f, output_f)

# Remove the temporary CSV file
os.remove(temp_csv)

print(f"Compressed CSV saved as {output_csv_zst}")

Compressed CSV saved as submissions.csv.zst
